# 필요한 모듈 설치 및 임포트

In [50]:
!pip install selenium==4.8.2

In [51]:
!pip install webdriver_manager

In [3]:
from bs4 import BeautifulSoup
import selenium
from selenium import webdriver

from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import numpy as np
import time
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import re
# driver.quit()
print(selenium.__version__)

4.8.2


# Selenium을 이용하여 브라우저 열기, 클릭, 페이지 이동, 닫기

## 가장 먼저 실행! 

In [4]:

chrome_options = Options()

chrome_options.add_experimental_option("detach", True)
chrome_options.add_experimental_option("excludeSwitches",["enable-logging"])

driver = webdriver.Chrome(options=chrome_options)

In [6]:
url = 'https://www.musinsa.com/app/codimap/lists?style_type=&tag_no=&brand=&display_cnt=60&list_kind=big&sort=comment_cnt&page=1'

# 무신사 코디맵 정보 크롤링

In [27]:
# options = webdriver.ChromeOptions()
# driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

driver = webdriver.Chrome(options=chrome_options)
driver.get(url)
driver.implicitly_wait(2)
try:
    driver.maximize_window()
except:
    pass

for weekend in range(2):
    # 페이지를 가장 아래까지 스크롤하는 방법

    interval = 1 # 1초에 한번씩 스크롤 내림

    # 현재 문서 높이를 가져와서 저장
    prev_height = driver.execute_script("return document.body.scrollHeight")

    # 반복 수행
    while True:
        # 스크롤을 가장 아래로 내림
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight)")

        # 페이지 로딩 대기
        time.sleep(interval)

        # 현재 문서 높이를 가져와서 저장
        curr_height = driver.execute_script("return document.body.scrollHeight")
        if curr_height == prev_height:
            break

        prev_height = curr_height

    # 해당 Xpath 정보를 확인할때까지 5초간 기다림, 5초후에도 해당 Xpath를 찾지 못하면 에러 반환
    # elem = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, '//*[@id="content"]/div[1]/ul/li[40]/a/div/img')))

    columns = ['codimap_category', 'codimap_title','codimap_date','views','comment_numbers']         
    values1 = []       
    list1=[]                                      # 리스트 정보를 담을 빈 리스트 생성
    soup = BeautifulSoup(driver.page_source, 'lxml')        # 현재 브라우저의 page의 html 정보를 lxml을 이용하여 가져오기
    data_rows = soup.find_all('li', attrs={'class':'style-list-item'}) # 해당 조건에 맞는 정보들을 list로 반환

    for i, row in enumerate(data_rows):                     # data_rows 리스트에 있는 값들과 인덱스를 차례대로 가져온다

        print('{0}번째 코디 크롤링'.format(i+1))
        blank = []
        c_category = row.find('span', attrs={'class':'style-list-information__text'})       # 카테고리 불러오기
        if c_category:                                                                    # 해당 조건에 맞는 정보가 하나라도 있을 경우:
            c_category = c_category.get_text().strip()                                          # html에서 텍스트만 가져온 뒤 공백 제거
            blank.append(c_category)
        else:                                                                       # 해당 조건에 맞는 정보가 하나도 없을 경우:
            blank.append('Something is wrong')
            print('{}번째 코디 카테고리 가져올때 문제발생'.format(i+1))
            continue

        c_title = row.find('strong', attrs={'class':'style-list-information__title'})      # 타이틀 불러오기
        if c_title:
            c_title = c_title.get_text().strip()
            blank.append(c_title)
        else:
            blank.append('Something is wrong')
            print('{}번째 코디 타이틀에 문제발생'.format(i+1))
            continue
        c_date = row.find_all('span', attrs={'class':'post-information__text'})        # 날짜, 조회수, 댓글 불러오기
        if c_date:
            c_date_texts = [element.get_text().strip() for element in c_date]
            blank.extend(c_date_texts)
        else:
            blank.append('Something is wrong')
            print('{}번째 웹툰평점 가져올때 문제발생'.format(i+1))
            continue

        values1.append(blank)
        print('---------------------------------------------------')
    df1 = pd.DataFrame(values1, columns = columns)
    list1.append(df1) # 최종값 list 형태로 반환

1번째 코디 크롤링
---------------------------------------------------
2번째 코디 크롤링
---------------------------------------------------
3번째 코디 크롤링
---------------------------------------------------
4번째 코디 크롤링
---------------------------------------------------
5번째 코디 크롤링
---------------------------------------------------
6번째 코디 크롤링
---------------------------------------------------
7번째 코디 크롤링
---------------------------------------------------
8번째 코디 크롤링
---------------------------------------------------
9번째 코디 크롤링
---------------------------------------------------
10번째 코디 크롤링
---------------------------------------------------
11번째 코디 크롤링
---------------------------------------------------
12번째 코디 크롤링
---------------------------------------------------
13번째 코디 크롤링
---------------------------------------------------
14번째 코디 크롤링
---------------------------------------------------
15번째 코디 크롤링
---------------------------------------------------
16번째 코디 크롤링
-------------------------------------

In [29]:
list1

[   codimap_category codimap_title codimap_date      views comment_numbers
 0               캐주얼        톤온톤 코디     21.07.02  조회 17,231           댓글 50
 1               캐주얼        블랙 매니아     21.02.23  조회 36,302           댓글 47
 2               캐주얼        오늘의 코디     22.03.22  조회 13,381           댓글 44
 3                댄디       추천 댄디 룩     21.09.15  조회 21,579           댓글 32
 4               캐주얼         릴랙스 룩     21.02.23  조회 24,013           댓글 25
 5               스트릿        힙하다 힙해     21.02.03  조회 36,065           댓글 24
 6                댄디        데이트 코디     21.12.24  조회 10,956           댓글 19
 7               캐주얼        심플한 코디     21.07.30  조회 41,341           댓글 18
 8                댄디       훈훈 남친 룩     21.03.24  조회 19,984           댓글 17
 9                댄디      서머 데이트 룩     21.07.26   조회 8,353           댓글 17
 10               댄디     남친 룩으로 제격     21.12.24  조회 10,173           댓글 17
 11              캐주얼      지금 입기 좋아     21.08.31  조회 20,879           댓글 16
 12              캐주얼     

In [30]:
df_1 = pd.concat(list1).reset_index(drop=True) # index 초기화
df_1.head(10)

,codimap_category,codimap_title,codimap_date,views,comment_numbers
0,캐주얼,톤온톤 코디,21.07.02,"조회 17,231",댓글 50
1,캐주얼,블랙 매니아,21.02.23,"조회 36,302",댓글 47
2,캐주얼,오늘의 코디,22.03.22,"조회 13,381",댓글 44
3,댄디,추천 댄디 룩,21.09.15,"조회 21,579",댓글 32
4,캐주얼,릴랙스 룩,21.02.23,"조회 24,013",댓글 25
5,스트릿,힙하다 힙해,21.02.03,"조회 36,065",댓글 24
6,댄디,데이트 코디,21.12.24,"조회 10,956",댓글 19
7,캐주얼,심플한 코디,21.07.30,"조회 41,341",댓글 18
8,댄디,훈훈 남친 룩,21.03.24,"조회 19,984",댓글 17
9,댄디,서머 데이트 룩,21.07.26,"조회 8,353",댓글 17


In [31]:
df_1.shape

(60, 5)

# Dataframe을 csv 파일로 저장

In [32]:
df_1.to_csv('무신사 크롤링.csv', encoding = 'utf-8-sig') # 한글이 들어간 정보는 글자가 깨질 수 있어 encoding을 다음과 같이 지정하여 저장